In [1]:
# this script does the main calculation for this anlaysis
# it finds an estimate of the # of people on medicare and with these medical devices
# who were affected by outages at each time point

In [2]:
# for each time period
# build a union from all outage polygons at that time
#     
# for each time period
# then for each union
# find intersect of zip and union
# divide area of union by area of zip
# multiply ratio by number of people in zip to get estimate
# add to zip obj {time period: estimate}

# drew from https://gis.stackexchange.com/questions/82935/ogr-layer-intersection

In [3]:
from osgeo import ogr
import os
import json
from geojson import Polygon, Feature, FeatureCollection, dump
from shutil import copyfile


In [4]:
# consts
driver = ogr.GetDriverByName('GeoJSON')
diry = 'data'
sub_diry = 'zips'

orig_fn = 'zips_data.json' # your input
copy_fn = 'zips_with_estimates.json' # ogr saves right to the source file, so make a copy first then use that to be safe
orig_path = os.path.join(diry, sub_diry, orig_fn)
copy_path = os.path.join(diry, sub_diry, copy_fn)
copyfile(orig_path, copy_path) # copying so it's easy to redo this

zipsSource = driver.Open(copy_path, 1) # 0 means read-only. 1 means writeable. we want to write, right?
zip_lyr = zipsSource.GetLayer() # this file has one layer, so it's the first and only. Layers are like layers in a shape file or gdb

# outage source dir
outage_dir = 'geojson'

In [5]:
copy_path # double check

'data/zips/zips_with_estimates.json'

In [6]:
def reduce_name(x): # produces short dt code I used for field names
    dt = x[7:-8]
    n = ''
    if len(dt) == 6:
        n = dt[:-1] + '0' + dt[-1:]
    else:
        n= dt
    return n

In [7]:
# test adding a new float field by adding an area for each zip code
# adding a field in ogr has several steps
fld_name = 'zip_area' # give it a name
fld_def = ogr.FieldDefn(fld_name, ogr.OFTReal) # define it with the name str and data type, here it's a floating point number
fld_def.SetWidth(32) # set width of field
zip_lyr.CreateField(fld_def) # add field to the layer, like adding a column to a table

# loop thru zips and add area to 'zip_area'
# filling a field with data also has several steps
for feat in zip_lyr: 
    geom = feat.GetGeometryRef() # get the geometry of each feature in zip layer, each feature is a zip code
    area = geom.GetArea() # calculates the area of the zip code
    print(area) # so you can see it working
    feat.SetField('zip_area', area ) # put the calculated area in the field for this feature
    zip_lyr.SetFeature(feat) # the feature you just worked on is a copy, replace the eature in the dataset with the copy
zip_lyr.ResetReading() # reset the loop so looping on the layer starts from 0 next time, else it starts at where the last loop ended (the end of the layer)

0.3009464449190193
0.00032469725690884715
0.0362311149916983
0.1194541844704522
0.030235862595243156
0.11416378519313554
0.007038672216076389
0.24832696073518445
0.18290939425222075
0.0014020758789614618
0.18892284262694098
0.13004988992623473
0.19146440305382625
0.0004802203032739389
0.33538952489594426
0.05616374366605116
0.06255694693673458
0.001501662150885097
0.05975322535183959
0.07193153838132124
0.0568731290515461
0.01117665292368929
0.007036693134606484
0.052026035308670525
0.009371746794877591
0.08251941392331767
0.04895330465116949
0.048810233762023876
0.020400002030996255
0.03270097978721127
0.009714050168792234
0.0636442641038934
0.0447439797249079
0.06840879655644441
0.0659105441954479
0.048990769121290266
0.01483916519069689
0.00770658063108072
0.12875623717858165
0.009678101469203143
0.004322066917715712
5.892499444496574e-06
7.091845472764063e-05
0.00020544844967504937
0.034211071769650936
0.12934210562713117
0.014790985148219802
0.0013174383615893771
0.152095826253890

In [8]:
for feat in zip_lyr: 
    print(feat.GetField("zip_area") )
zip_lyr.ResetReading() # see we do this every time, and beforehand as well when we want to be extra sure
# sanity check, make sure it looks good

0.3009464449190193
0.00032469725690884715
0.0362311149916983
0.1194541844704522
0.030235862595243156
0.11416378519313554
0.007038672216076389
0.24832696073518445
0.18290939425222075
0.0014020758789614618
0.18892284262694098
0.13004988992623473
0.19146440305382625
0.0004802203032739389
0.33538952489594426
0.05616374366605116
0.06255694693673458
0.001501662150885097
0.05975322535183959
0.07193153838132124
0.0568731290515461
0.01117665292368929
0.007036693134606484
0.052026035308670525
0.009371746794877591
0.08251941392331767
0.04895330465116949
0.048810233762023876
0.020400002030996255
0.03270097978721127
0.009714050168792234
0.0636442641038934
0.0447439797249079
0.06840879655644441
0.0659105441954479
0.048990769121290266
0.01483916519069689
0.00770658063108072
0.12875623717858165
0.009678101469203143
0.004322066917715712
5.892499444496574e-06
7.091845472764063e-05
0.00020544844967504937
0.034211071769650936
0.12934210562713117
0.014790985148219802
0.0013174383615893771
0.152095826253890

In [9]:
schema = []
ldefn = zip_lyr.GetLayerDefn()
for n in range(ldefn.GetFieldCount()):
    fdefn = ldefn.GetFieldDefn(n)
    schema.append(fdefn.name)
print(schema) # should be a list of the fields (aside from geometry) attached to each zip code feature

['STATE', 'Medicare_Benes', 'Power_Dependent_Devices_DME', 'Zip_Code', 'Zip_Code_Recode', 'zip_area']


In [10]:
# get list of outage files
outage_files = []
outage_names = []
outage_path = os.path.join(diry, outage_dir)
for fn in os.listdir(outage_path):
    outage_files.append(fn)
    outage_names.append(fn[:-8])
print(outage_files) # just checking
print(outage_names) 

['outages1008_12.geojson', 'outages1008_18.geojson', 'outages1009_0.geojson', 'outages1009_12.geojson', 'outages1009_18.geojson', 'outages1009_6.geojson', 'outages1010_0.geojson', 'outages1010_12.geojson', 'outages1010_18.geojson', 'outages1010_6.geojson', 'outages1026_0.geojson', 'outages1026_12.geojson', 'outages1026_18.geojson', 'outages1026_6.geojson', 'outages1027_0.geojson', 'outages1027_12.geojson', 'outages1027_18.geojson', 'outages1027_6.geojson', 'outages1028_0.geojson', 'outages1028_12.geojson', 'outages1028_18.geojson', 'outages1028_6.geojson', 'outages1029_0.geojson', 'outages1029_12.geojson', 'outages1029_18.geojson', 'outages1029_6.geojson', 'outages1030_0.geojson', 'outages1030_12.geojson', 'outages1030_18.geojson', 'outages1030_6.geojson', 'outages1031_0.geojson', 'outages1031_12.geojson', 'outages1031_18.geojson', 'outages1031_6.geojson', 'outages1120_0.geojson', 'outages1120_12.geojson', 'outages1120_18.geojson', 'outages1120_6.geojson', 'outages1121_0.geojson', 'out

In [11]:
# create estimate fields for all the outage data
for nm in outage_names:
    name = nm[7:] # just the time and date
    if len(name) == 6:
        name = name[:5] + '0' + name[5] # pad with zeroes
    e_name = name + '_est'
    efld_def = ogr.FieldDefn(e_name, ogr.OFTReal) # again define field, etc.
    efld_def.SetWidth(32)
    zip_lyr.CreateField(efld_def)
    print(e_name)
# creates fields like
# '1008_12_est'

1008_12_est
1008_18_est
1009_00_est
1009_12_est
1009_18_est
1009_06_est
1010_00_est
1010_12_est
1010_18_est
1010_06_est
1026_00_est
1026_12_est
1026_18_est
1026_06_est
1027_00_est
1027_12_est
1027_18_est
1027_06_est
1028_00_est
1028_12_est
1028_18_est
1028_06_est
1029_00_est
1029_12_est
1029_18_est
1029_06_est
1030_00_est
1030_12_est
1030_18_est
1030_06_est
1031_00_est
1031_12_est
1031_18_est
1031_06_est
1120_00_est
1120_12_est
1120_18_est
1120_06_est
1121_00_est
1121_12_est
1121_18_est
1121_06_est


In [12]:
# check schema again
schema = []
ldefn = zip_lyr.GetLayerDefn()
for n in range(ldefn.GetFieldCount()):
    fdefn = ldefn.GetFieldDefn(n)
    schema.append(fdefn.name)
print(schema) # should see 42 est fields

['STATE', 'Medicare_Benes', 'Power_Dependent_Devices_DME', 'Zip_Code', 'Zip_Code_Recode', 'zip_area', '1008_12_est', '1008_18_est', '1009_00_est', '1009_12_est', '1009_18_est', '1009_06_est', '1010_00_est', '1010_12_est', '1010_18_est', '1010_06_est', '1026_00_est', '1026_12_est', '1026_18_est', '1026_06_est', '1027_00_est', '1027_12_est', '1027_18_est', '1027_06_est', '1028_00_est', '1028_12_est', '1028_18_est', '1028_06_est', '1029_00_est', '1029_12_est', '1029_18_est', '1029_06_est', '1030_00_est', '1030_12_est', '1030_18_est', '1030_06_est', '1031_00_est', '1031_12_est', '1031_18_est', '1031_06_est', '1120_00_est', '1120_12_est', '1120_18_est', '1120_06_est', '1121_00_est', '1121_12_est', '1121_18_est', '1121_06_est']


In [13]:
# loop through and create outage layers in object
outage_sources = {}
outage_layers = {}
for fn in outage_files:
    outage_path = os.path.join(diry, outage_dir, fn)
    print(outage_path)
    outage_sources[fn] = driver.Open(outage_path, 0) # 0 for read-only. Don't want to write to these
    outage_layers[fn] = outage_sources[fn].GetLayer() # first layer is only layer again


data/geojson/outages1008_12.geojson
data/geojson/outages1008_18.geojson
data/geojson/outages1009_0.geojson
data/geojson/outages1009_12.geojson
data/geojson/outages1009_18.geojson
data/geojson/outages1009_6.geojson
data/geojson/outages1010_0.geojson
data/geojson/outages1010_12.geojson
data/geojson/outages1010_18.geojson
data/geojson/outages1010_6.geojson
data/geojson/outages1026_0.geojson
data/geojson/outages1026_12.geojson
data/geojson/outages1026_18.geojson
data/geojson/outages1026_6.geojson
data/geojson/outages1027_0.geojson
data/geojson/outages1027_12.geojson
data/geojson/outages1027_18.geojson
data/geojson/outages1027_6.geojson
data/geojson/outages1028_0.geojson
data/geojson/outages1028_12.geojson
data/geojson/outages1028_18.geojson
data/geojson/outages1028_6.geojson
data/geojson/outages1029_0.geojson
data/geojson/outages1029_12.geojson
data/geojson/outages1029_18.geojson
data/geojson/outages1029_6.geojson
data/geojson/outages1030_0.geojson
data/geojson/outages1030_12.geojson
data/

In [14]:
# make a union of all features in outage layers
unions = None
outage_unions = {}
# loop through outage files
for fn in outage_files:
    outage_lyr = None
    outage_lyr = outage_layers[fn]
    outage_lyr.ResetReading()
    union = None
    union = ogr.Geometry(3)
    # loop thru features of outage files
    for feat in outage_lyr:
        geom =feat.GetGeometryRef() # grab each feat
        union = union.Union(geom)        
    outage_unions[fn] = union
    print(fn)

# make sure outage layers are reset
for fn in outage_files:
    outage_lyr = None
    outage_lyr = outage_layers[fn]
    outage_lyr.ResetReading() # we don't use these again but maybe we'll have to rerun this cell

outages1008_12.geojson
outages1008_18.geojson
outages1009_0.geojson
outages1009_12.geojson
outages1009_18.geojson
outages1009_6.geojson
outages1010_0.geojson
outages1010_12.geojson
outages1010_18.geojson
outages1010_6.geojson
outages1026_0.geojson
outages1026_12.geojson
outages1026_18.geojson
outages1026_6.geojson
outages1027_0.geojson
outages1027_12.geojson
outages1027_18.geojson
outages1027_6.geojson
outages1028_0.geojson
outages1028_12.geojson
outages1028_18.geojson
outages1028_6.geojson
outages1029_0.geojson
outages1029_12.geojson
outages1029_18.geojson
outages1029_6.geojson
outages1030_0.geojson
outages1030_12.geojson
outages1030_18.geojson
outages1030_6.geojson
outages1031_0.geojson
outages1031_12.geojson
outages1031_18.geojson
outages1031_6.geojson
outages1120_0.geojson
outages1120_12.geojson
outages1120_18.geojson
outages1120_6.geojson
outages1121_0.geojson
outages1121_12.geojson
outages1121_18.geojson
outages1121_6.geojson


In [15]:
# at this point, we should have a zip layer and a list of
# outage uions, which are geometeries
# and the zipcode layer should have fields set up ready to recieve 
# results of the calculations

In [16]:
# sanity checks
print(len(outage_unions))
print(len(outage_files))
# should be 42 and 42

42
42


In [17]:
# this loop creates those estimates
zip_lyr.ResetReading()
for z_feat in zip_lyr: # loop through zip code layer
    z_area = z_feat.GetField("zip_area") # zip code area
    z_geom = z_feat.GetGeometryRef() # geometry of zip code
    z_code = z_feat.GetField("Zip_Code")
    z_num = z_feat.GetField("Power_Dependent_Devices_DME")
    print(z_code)
    # loop thru unions, 
    for u_key in outage_unions.keys():
        print(u_key)
        # get union data we need
        u = outage_unions[u_key] # this is the union geometry
        u_est_key = reduce_name(u_key) + '_est' # keys in ziplyr
        # calculate intersect 
        zu_int = z_geom.Intersection(u) # intersection geometry
        # and area of intersect an  
        int_area = zu_int.GetArea() # get area of intersectio
        int_aratio = int_area / z_area # get proportion of overlap
        est = int_aratio * z_num # estimate number affected
        # add estimate to field in zip layer
        z_feat.SetField(u_est_key, est ) 
        zip_lyr.SetFeature(z_feat) 
    print('-----------------------\n\n\n')
zip_lyr.ResetReading() # reset so it can loop again


96056
outages1008_12.geojson
outages1008_18.geojson
outages1009_0.geojson
outages1009_12.geojson
outages1009_18.geojson
outages1009_6.geojson
outages1010_0.geojson
outages1010_12.geojson
outages1010_18.geojson
outages1010_6.geojson
outages1026_0.geojson
outages1026_12.geojson
outages1026_18.geojson
outages1026_6.geojson
outages1027_0.geojson
outages1027_12.geojson
outages1027_18.geojson
outages1027_6.geojson
outages1028_0.geojson
outages1028_12.geojson
outages1028_18.geojson
outages1028_6.geojson
outages1029_0.geojson
outages1029_12.geojson
outages1029_18.geojson
outages1029_6.geojson
outages1030_0.geojson
outages1030_12.geojson
outages1030_18.geojson
outages1030_6.geojson
outages1031_0.geojson
outages1031_12.geojson
outages1031_18.geojson
outages1031_6.geojson
outages1120_0.geojson
outages1120_12.geojson
outages1120_18.geojson
outages1120_6.geojson
outages1121_0.geojson
outages1121_12.geojson
outages1121_18.geojson
outages1121_6.geojson
-----------------------



96068
outages1008_12.

outages1027_12.geojson
outages1027_18.geojson
outages1027_6.geojson
outages1028_0.geojson
outages1028_12.geojson
outages1028_18.geojson
outages1028_6.geojson
outages1029_0.geojson
outages1029_12.geojson
outages1029_18.geojson
outages1029_6.geojson
outages1030_0.geojson
outages1030_12.geojson
outages1030_18.geojson
outages1030_6.geojson
outages1031_0.geojson
outages1031_12.geojson
outages1031_18.geojson
outages1031_6.geojson
outages1120_0.geojson
outages1120_12.geojson
outages1120_18.geojson
outages1120_6.geojson
outages1121_0.geojson
outages1121_12.geojson
outages1121_18.geojson
outages1121_6.geojson
-----------------------



96136
outages1008_12.geojson
outages1008_18.geojson
outages1009_0.geojson
outages1009_12.geojson
outages1009_18.geojson
outages1009_6.geojson
outages1010_0.geojson
outages1010_12.geojson
outages1010_18.geojson
outages1010_6.geojson
outages1026_0.geojson
outages1026_12.geojson
outages1026_18.geojson
outages1026_6.geojson
outages1027_0.geojson
outages1027_12.geojso

outages1028_6.geojson
outages1029_0.geojson
outages1029_12.geojson
outages1029_18.geojson
outages1029_6.geojson
outages1030_0.geojson
outages1030_12.geojson
outages1030_18.geojson
outages1030_6.geojson
outages1031_0.geojson
outages1031_12.geojson
outages1031_18.geojson
outages1031_6.geojson
outages1120_0.geojson
outages1120_12.geojson
outages1120_18.geojson
outages1120_6.geojson
outages1121_0.geojson
outages1121_12.geojson
outages1121_18.geojson
outages1121_6.geojson
-----------------------



96115
outages1008_12.geojson
outages1008_18.geojson
outages1009_0.geojson
outages1009_12.geojson
outages1009_18.geojson
outages1009_6.geojson
outages1010_0.geojson
outages1010_12.geojson
outages1010_18.geojson
outages1010_6.geojson
outages1026_0.geojson
outages1026_12.geojson
outages1026_18.geojson
outages1026_6.geojson
outages1027_0.geojson
outages1027_12.geojson
outages1027_18.geojson
outages1027_6.geojson
outages1028_0.geojson
outages1028_12.geojson
outages1028_18.geojson
outages1028_6.geojson

outages1028_12.geojson
outages1028_18.geojson
outages1028_6.geojson
outages1029_0.geojson
outages1029_12.geojson
outages1029_18.geojson
outages1029_6.geojson
outages1030_0.geojson
outages1030_12.geojson
outages1030_18.geojson
outages1030_6.geojson
outages1031_0.geojson
outages1031_12.geojson
outages1031_18.geojson
outages1031_6.geojson
outages1120_0.geojson
outages1120_12.geojson
outages1120_18.geojson
outages1120_6.geojson
outages1121_0.geojson
outages1121_12.geojson
outages1121_18.geojson
outages1121_6.geojson
-----------------------



95983
outages1008_12.geojson
outages1008_18.geojson
outages1009_0.geojson
outages1009_12.geojson
outages1009_18.geojson
outages1009_6.geojson
outages1010_0.geojson
outages1010_12.geojson
outages1010_18.geojson
outages1010_6.geojson
outages1026_0.geojson
outages1026_12.geojson
outages1026_18.geojson
outages1026_6.geojson
outages1027_0.geojson
outages1027_12.geojson
outages1027_18.geojson
outages1027_6.geojson
outages1028_0.geojson
outages1028_12.geojso

outages1120_6.geojson
outages1121_0.geojson
outages1121_12.geojson
outages1121_18.geojson
outages1121_6.geojson
-----------------------



96122
outages1008_12.geojson
outages1008_18.geojson
outages1009_0.geojson
outages1009_12.geojson
outages1009_18.geojson
outages1009_6.geojson
outages1010_0.geojson
outages1010_12.geojson
outages1010_18.geojson
outages1010_6.geojson
outages1026_0.geojson
outages1026_12.geojson
outages1026_18.geojson
outages1026_6.geojson
outages1027_0.geojson
outages1027_12.geojson
outages1027_18.geojson
outages1027_6.geojson
outages1028_0.geojson
outages1028_12.geojson
outages1028_18.geojson
outages1028_6.geojson
outages1029_0.geojson
outages1029_12.geojson
outages1029_18.geojson
outages1029_6.geojson
outages1030_0.geojson
outages1030_12.geojson
outages1030_18.geojson
outages1030_6.geojson
outages1031_0.geojson
outages1031_12.geojson
outages1031_18.geojson
outages1031_6.geojson
outages1120_0.geojson
outages1120_12.geojson
outages1120_18.geojson
outages1120_6.geojson

...

outages1009_6.geojson
outages1010_0.geojson
outages1010_12.geojson
outages1010_18.geojson
outages1010_6.geojson
outages1026_0.geojson
outages1026_12.geojson
outages1026_18.geojson
outages1026_6.geojson
outages1027_0.geojson
outages1027_12.geojson
outages1027_18.geojson
outages1027_6.geojson
outages1028_0.geojson
outages1028_12.geojson
outages1028_18.geojson
outages1028_6.geojson
outages1029_0.geojson
outages1029_12.geojson
outages1029_18.geojson
outages1029_6.geojson
outages1030_0.geojson
outages1030_12.geojson
outages1030_18.geojson
outages1030_6.geojson
outages1031_0.geojson
outages1031_12.geojson
outages1031_18.geojson
outages1031_6.geojson
outages1120_0.geojson
outages1120_12.geojson
outages1120_18.geojson
outages1120_6.geojson
outages1121_0.geojson
outages1121_12.geojson
outages1121_18.geojson
outages1121_6.geojson
-----------------------



95480
outages1008_12.geojson
outages1008_18.geojson
outages1009_0.geojson
outages1009_12.geojson
outages1009_18.geojson
outages1009_6.geojson

outages1009_6.geojson
outages1010_0.geojson
outages1010_12.geojson
outages1010_18.geojson
outages1010_6.geojson
outages1026_0.geojson
outages1026_12.geojson
outages1026_18.geojson
outages1026_6.geojson
outages1027_0.geojson
outages1027_12.geojson
outages1027_18.geojson
outages1027_6.geojson
outages1028_0.geojson
outages1028_12.geojson
outages1028_18.geojson
outages1028_6.geojson
outages1029_0.geojson
outages1029_12.geojson
outages1029_18.geojson
outages1029_6.geojson
outages1030_0.geojson
outages1030_12.geojson
outages1030_18.geojson
outages1030_6.geojson
outages1031_0.geojson
outages1031_12.geojson
outages1031_18.geojson
outages1031_6.geojson
outages1120_0.geojson
outages1120_12.geojson
outages1120_18.geojson
outages1120_6.geojson
outages1121_0.geojson
outages1121_12.geojson
outages1121_18.geojson
outages1121_6.geojson
-----------------------



95430
outages1008_12.geojson
outages1008_18.geojson
outages1009_0.geojson
outages1009_12.geojson
outages1009_18.geojson
outages1009_6.geojson

outages1031_6.geojson
outages1120_0.geojson
outages1120_12.geojson
outages1120_18.geojson
outages1120_6.geojson
outages1121_0.geojson
outages1121_12.geojson
outages1121_18.geojson
outages1121_6.geojson
-----------------------



93604
outages1008_12.geojson
outages1008_18.geojson
outages1009_0.geojson
outages1009_12.geojson
outages1009_18.geojson
outages1009_6.geojson
outages1010_0.geojson
outages1010_12.geojson
outages1010_18.geojson
outages1010_6.geojson
outages1026_0.geojson
outages1026_12.geojson
outages1026_18.geojson
outages1026_6.geojson
outages1027_0.geojson
outages1027_12.geojson
outages1027_18.geojson
outages1027_6.geojson
outages1028_0.geojson
outages1028_12.geojson
outages1028_18.geojson
outages1028_6.geojson
outages1029_0.geojson
outages1029_12.geojson
outages1029_18.geojson
outages1029_6.geojson
outages1030_0.geojson
outages1030_12.geojson
outages1030_18.geojson
outages1030_6.geojson
outages1031_0.geojson
outages1031_12.geojson
outages1031_18.geojson
outages1031_6.geojson

outages1029_0.geojson
outages1029_12.geojson
outages1029_18.geojson
outages1029_6.geojson
outages1030_0.geojson
outages1030_12.geojson
outages1030_18.geojson
outages1030_6.geojson
outages1031_0.geojson
outages1031_12.geojson
outages1031_18.geojson
outages1031_6.geojson
outages1120_0.geojson
outages1120_12.geojson
outages1120_18.geojson
outages1120_6.geojson
outages1121_0.geojson
outages1121_12.geojson
outages1121_18.geojson
outages1121_6.geojson
-----------------------



93638
outages1008_12.geojson
outages1008_18.geojson
outages1009_0.geojson
outages1009_12.geojson
outages1009_18.geojson
outages1009_6.geojson
outages1010_0.geojson
outages1010_12.geojson
outages1010_18.geojson
outages1010_6.geojson
outages1026_0.geojson
outages1026_12.geojson
outages1026_18.geojson
outages1026_6.geojson
outages1027_0.geojson
outages1027_12.geojson
outages1027_18.geojson
outages1027_6.geojson
outages1028_0.geojson
outages1028_12.geojson
outages1028_18.geojson
outages1028_6.geojson
outages1029_0.geojson

outages1027_6.geojson
outages1028_0.geojson
outages1028_12.geojson
outages1028_18.geojson
outages1028_6.geojson
outages1029_0.geojson
outages1029_12.geojson
outages1029_18.geojson
outages1029_6.geojson
outages1030_0.geojson
outages1030_12.geojson
outages1030_18.geojson
outages1030_6.geojson
outages1031_0.geojson
outages1031_12.geojson
outages1031_18.geojson
outages1031_6.geojson
outages1120_0.geojson
outages1120_12.geojson
outages1120_18.geojson
outages1120_6.geojson
outages1121_0.geojson
outages1121_12.geojson
outages1121_18.geojson
outages1121_6.geojson
-----------------------



95018
outages1008_12.geojson
outages1008_18.geojson
outages1009_0.geojson
outages1009_12.geojson
outages1009_18.geojson
outages1009_6.geojson
outages1010_0.geojson
outages1010_12.geojson
outages1010_18.geojson
outages1010_6.geojson
outages1026_0.geojson
outages1026_12.geojson
outages1026_18.geojson
outages1026_6.geojson
outages1027_0.geojson
outages1027_12.geojson
outages1027_18.geojson
outages1027_6.geojson

In [18]:
# see estimates for first date time
for feat in zip_lyr: 
    print(feat.GetField("1010_12_est") )
zip_lyr.ResetReading()

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.1608883854675591
0.013968074786692685
0.0
0.0
1.5927316606913326e-05
0.6088577666750553
0.6982860346150487
0.0
1.4627031069413883
0.34031136566458575
0.2654900465509692
0.0
0.0
0.0
0.0
0.0
0.0
0.0
34.86956807930086
26.232455393752886
59.38263309895314
10.999999999993523
3.8365655516843127
7.938798362383706
0.0
0.0
22.13979694579615
9.669419522738071
0.867404885103827
4.5981025968691105
0.0
0.0
0.0
0.0
0.0
0.0
0.0
7.221649587836898
0.0796957207634526
30.541651130117938
2.725779274627535
0.0
3.13890076846409
0.3360890460087551
0.0
110.94077552148887
0.0
0.0
0.0
0.0
225.60261294120642
302.3938995974952
255.91060677316088
0.0
0.9051208800315272
4.121081191616239
0.0
37.61652748087606
9.500051367234418
349.4764942232512
53.45326706703336
0.0
0.0
0.0
0.0
0.0
0.0
104.02402973168839
0.0
0.0
0.0
0.0
0.0
0.0
0.0009388703820749211
0.0
0.01719073490619806
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.8931168392405303
0.0

In [19]:
# should do this earlier, but here we can get spatial reference info for these data sets
# it's all in decimal degrees lat long anyway
srs = zip_lyr.GetSpatialRef() # grabs spatial reference
print('zip spatial ref')
print(srs)
print('outage spatial refs')
for fn in outage_files:
    outage_lyr = None
    outage_lyr = outage_layers[fn]
    print( outage_lyr.GetSpatialRef() )

# they match

zip spatial ref
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]
outage spatial refs
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["E

In [20]:
# clearing the reference to the zips file closes it and saves what was added
zipsSource.Destroy()
zipsSource = None 
# the data file should have the same name as the copy file at start